In [9]:
from pathlib import Path
import numpy as np

In [20]:
root_path = Path("/media/hdd1/i-care")
ae_feat_path = root_path / "ae-features"
man_feat_path = root_path / "features"
combined_feat_path = root_path / "combined-features"

In [23]:
for patient_dir in list(ae_feat_path.iterdir())[:1]:
    (combined_feat_path / patient_dir.name).mkdir(parents=True, exist_ok=True)

    for ae_file in patient_dir.iterdir():
        # Extract hour from the filename
        hour = ae_file.stem.split("_")[1]
        # Find corresponding manual feature file
        man_file_pattern = f"{patient_dir.name}_*_{hour}_features.npy"
        man_file = list(man_feat_path.joinpath(patient_dir.name).glob(man_file_pattern))

        if man_file:
            man_file = man_file[0]
            # Load feature arrays
            ae_feat = np.load(ae_file)
            man_feat = np.load(man_file)
            # Truncate to the shortest sample
            min_sample = min(ae_feat.shape[0], man_feat.shape[0])
            ae_feat = ae_feat[:min_sample]
            man_feat = man_feat[:min_sample]
            # Concatenate features
            combined_feat = np.hstack((man_feat, ae_feat))
            # Save combined features
            combined_filename = f"{patient_dir.name}_{hour}_features.npy"
            np.save(combined_feat_path / patient_dir.name / combined_filename, combined_feat)